# Ingestando embeddings a Pinecone

In [1]:
import pinecone
import os, getpass
import tiktoken

/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")

## Creando un index en Pinecone

In [3]:
env = "gcp-starter"
index_name = "knowledge-base-eliminatorias"
dimension = 1536

pinecone.init(api_key = api_key, environment = env)

if index_name not in pinecone.list_indexes():

    pinecone.create_index(
        index_name, 
        dimension = dimension
    )

    print("Index " + index_name + " creado con éxito en Pinecone")

Index knowledge-base-eliminatorias creado con éxito en Pinecone


## Generando fragmentos de sitios web

![Imagen](https://imgmedia.larepublica.pe/640x371/larepublica/original/2023/10/17/652f24917388f967a923a805.webp)

In [14]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader(
    [
    "https://www.marca.com/co/2023/10/17/652e070f22601d73648b4585.html", 
    "https://hiraoka.com.pe/blog/post/eliminatorias-sudamericanas-mundial-2026-calendario-partidos-y-fechas"
    ]
)
data = loader.load()

#Genera varios fragmentos de 400 tokens
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 500, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(data)

Created a chunk of size 1216, which is longer than the specified 500
Created a chunk of size 1299, which is longer than the specified 500
Created a chunk of size 1299, which is longer than the specified 500
Created a chunk of size 2285, which is longer than the specified 500


In [15]:
len(docs), docs

(29,
 [Document(page_content='Tabla de posiciones Eliminatorias Mundial 2026: Clasificación y resultados de la cuarta fecha en Sudamérica | Marca \n\n\n \n\n \n\n\n \n \n\n\n  \n\n \n\n\nEs noticia: Athletic - CeltaLeganés - LevanteConvocatoria de la FuenteEspaña - PoloniaPedro AcostaBaldeAthletic Club - Celta TVFernando AlonsoGironaCanasta Felipe ReyesBarcelona - Estrella Roja TVVolkanovskiMotoGP Malasia horarioPortero First DatesIllias AkhomachAlba CarrilloExluchadora OnlyFansLawfareMono quiméricoBlack FridayInvestidura Pedro SánchezNominados GH VIPCándido Conde-Pumpido First DatesEl Hormiguero invitadosInvitados PasapalabraPrecio luz hoyCalculadora ChampionsResultados fútbolCalendario F1 \n\nPortada de Marca\n\n\nEdición seleccionada\nES \n\n\nEdiciones:\n\nES (España)\n\nEN (English)\n\n\nMX (México)\n\n\nUS (USA)\n\nEscoge tu edición de Marca.com favorita\n\n\nSiempre que entres en Marca.com se cargará esta edición\n\nES (España)\n\n \n\nEN (English)\n\n \n\nMX (México)\n\n \n\nUS

In [16]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingrese la API Key de OpenAI : ")

## Cargando datos a Pinecone

In [17]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)